In [7]:
%load_ext autoreload
%autoreload 2
import numpy as np
import NuLattice.smearedInteraction as smearI
import NuLattice.references as ref
import NuLattice.FCI.few_body_diagonalization as fbd
import NuLattice.lattice as lat
from scipy.sparse.linalg import eigsh as arpack_eigsh
import matplotlib.pyplot as plt
import NuLattice.CCM.coupled_cluster as ccm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
myL= 5
bpi = 0.7
sL = 0.08
sNL = 0.08
a = 1.0 / 100.0
c0 = -1.85e-05/ (a ** 2)
lattice = lat.get_lattice(myL)
tKin = smearI.tKin(myL, 3, a)

In [9]:
ope = smearI.onePionEx(myL,bpi, a,lattice, verbose=True, mult = 1)

Calculating f_SS...Done
Calculating Densities...Done
Generating Interaction...
Compressing interaction...Done
Interaction Generated
Converting to List...Done


In [10]:
# Compute the deuteron
# additive quantum numbers
numpart=2 # number of nucleons
tz = 0    # twice the value of isospin projection
sz = 2    # twice the value of spin projection
my_basis = lat.get_sp_basis(myL)
nstat =  len(my_basis)

# get two-body basis as a dictionary for lookup
H2_lookup = fbd.get_many_body_states(my_basis, numpart, total_tz=tz, total_sz=None)
print("matrix dimension:", len(H2_lookup))

# make scipy.sparse.csr_matrix for kinetic energy 
T2_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, tKin, nstat)
print("kinetic energy done")

# make scipy.sparse.csr_matrix for 2-body interactions 
OPE_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, ope, nstat)
print("2-body interaction done")
# add all into Hamiltonian
H2_csr_mat = T2_csr_mat + OPE_csr_mat

# compute lowest eigenvalue(s)
k_eig=10 # number of eigenvalues
vals, vecs = arpack_eigsh(H2_csr_mat, k=k_eig, which='SA')
print("Energy (MeV):", vals[0])

matrix dimension: 62500
kinetic energy done
2-body interaction done
Energy (MeV): -0.0001219652144306102


In [11]:
inter = smearI.shortRangeV(lattice,myL, sL, sNL, c0, a, verbose = True)

Generating Densities...Done
Generating rho * f_SL...Done
Generating Interaction...
Compressing interaction...Done
Compressing interaction...Done
Compressing interaction...Done
Compressing interaction...Done
Compressing interaction...Done
Compressing interaction...Done
Compressing interaction...Done
Compressing interaction...Done
Interaction Generated
Converting to List...Done


In [12]:
# make scipy.sparse.csr_matrix for 2-body interactions
V2_csr_mat = fbd.get_csr_matrix_scalar_op(H2_lookup, inter, nstat)
print("2-body interaction done")

# add all into Hamiltonian
H2_csr_mat = T2_csr_mat + V2_csr_mat + OPE_csr_mat

# compute lowest eigenvalue(s)
k_eig=10 # number of eigenvalues
vals, vecs = arpack_eigsh(H2_csr_mat, k=k_eig, which='SA')
print("Energies (MeV):", vals[0])

2-body interaction done
Energies (MeV): -3.3763486465964205
